### importo l'ambiente TAXI da gymnasium creo un envoirement, lo resetto e con il comando render visualizzo l'ambiente  

In [1]:
import gymnasium as gym
import random 
from IPython.display import clear_output
from time import sleep
import numpy as np


env = gym.make("Taxi-v3", render_mode="ansi").env

env.reset()
print(env.render())
# Le 4 lettere rappresentano la postazione dove puo essere il passeggero e allo stesso tempo dove puo essere portato il giallo e' il taxi

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




### Esplorazione dell'ambiente per mappare stati e azioni

In [2]:
env.reset()
env.render()

print("Action Space{}".format(env.action_space) )
print("Che sono le azioni possibili")
print("State Space {}".format(env.observation_space))
print("Che sono i stati possibili")

Action SpaceDiscrete(6)
Che sono le azioni possibili
State Space Discrete(500)
Che sono i stati possibili


### manipolazioni all'interno di uno stato 
#### don decode decodifichiamo lo stato con le variabili che lo definiscono 

#### creazione del mio stato partendo da determinate condizioni nelle variabili

In [3]:
print(env.s, '\n')
for el in env.decode(env.s):
    print(el)

print('\n imposto uno stato manualmente')
state = env.encode(3, 3, 1, 0)
print("State:", state, '\n')

env.s = state
print(env.render())

84 

0
4
1
0

 imposto uno stato manualmente
State: 364 

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




/home/claudio/Scrivania/Project/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.s to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.s` for environment variables or `env.get_wrapper_attr('s')` that will search the reminding wrappers.
  logger.warn(
/home/claudio/Scrivania/Project/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.decode to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.decode` for environment variables or `env.get_wrapper_attr('decode')` that will search the reminding wrappers.
  logger.warn(
/home/claudio/Scrivania/Project/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.encode to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.encode` for environment v

### Esplorazione reward table 
#### e' un dizionario strutturato con action : [(probabilty, nextstate, reward, done)]

In [4]:
env.P[364]

/home/claudio/Scrivania/Project/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


{0: [(1.0, 464, -1, False)],
 1: [(1.0, 264, -1, False)],
 2: [(1.0, 384, -1, False)],
 3: [(1.0, 364, -1, False)],
 4: [(1.0, 364, -10, False)],
 5: [(1.0, 364, -10, False)]}

### Simulazione random dell'agente che si muove nell'ambiente

In [5]:
env.s = 322

steps, penalties, reward = 0, 0, 0
frames = []
done = False
while not done:
    action = env.action_space.sample()
    state, reward , terminated, truncated, info = env.step(action)
    if reward ==-10:
        penalties += 1

    frames.append({
        'frame' : env.render(),
        'state' : state,
        'action': action,
        'reward' : reward
        }
                 )

    steps += 1
    done = terminated or truncated 

print("steps: {}".format(steps))
print("penalita': {}".format(penalties))

steps: 2562
penalita': 824


### Animazione della simulazione

In [6]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"step       :{i+1}")
        print(f"stato      :{frame['state']}")
        print(f"azione     :{frame['action']}")
        print(f"ricompensa :{frame['reward']}")
        sleep(.0001)

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

step       :1500
stato      :144
azione     :4
ricompensa :-10


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



### Instanzio una q-table random

In [7]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

### Addestramento dell'agente

In [8]:
gamma = 0.6  # Fattore di sconto 
epsilon = 0.1  # Fattore di esplorazione

all_epochs = []
all_penalties = []

#Per ogni esempio 
for i in range(1, 10001):
    state = env.reset()[0]  #Reset ambiente

    #Inizializzazione Step, penalita e ricompense 
    steps, penalites, reward = 0,0,0
    #done = False == Episodio non completo 
    done = False

    #Finche l'episodio non sara completo 
    while not done:
        # Numero casuale tra 0 e 1 e se e' minore di epsilon ossia del 10% effettuera un azione casuale ossia fara esplorazione
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            # Mentre 9 /10  volte effettura lo sfruttamento ossia utilizzera cio che gli viene consigliato 
            action = np.argmax(q_table[state]) # Sulla q-tabel nella riga dello stato attuale ho un elenco di colonne e prendo 
                                               # quella con il valore piu alto ossia id della colonna con il valore piu alto 

        next_state, reward, terminated, truncated, info = env.step(action)  # Eseguiamo l'azione precedentemente determinata
        old_value = q_table[state, action] #Sara l'azione eseguita nello stato precedente preso dalla q-table
        next_max = np.max(q_table[next_state]) # prendo la Q massima del nuovo stato dove sono entrato 

        new_value = reward + gamma * next_max # Applico Belman e dico che il valore di Q-table e' 
                                              # Il reward che ho ottenuto + il fattore di sconto * il massimo reward successivo allo 
                                              # allo stato attuale 
        # Lo implemento nella Q-table
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        steps += 1
        done = terminated or truncated 

    # una volta chiuso l'episodio 
    if i % 100 == 0:
        clear_output(wait= True)
        print(f"episodio:{i}")

print("terminato\n")

episodio:10000
terminato



### Al termine dei nostri 10000 episodi la nostra Q-table e' addestrata 

In [9]:
q_table[224]

array([ -2.4936525 ,  -2.4936525 ,  -2.48236806,  -2.4936525 ,
       -11.48942084, -11.48942084])

### Test dell'agente 

In [10]:
total_steps, total_penalties = 0, 0
episodies = 100 

for _ in range(episodies):
    state = env.reset()[0]   #Stato di partenza uno stato casuale 
    steps, penalties, reward = 0, 0, 0

    done = False 

    while not done:
        action = np.argmax(q_table[state])
        state, reward, termionated, truncated, info = env.step(action)  

        if reward == -10:
            penalties += 1

        steps += 1 

        done = truncated or terminated

    total_penalties += penalties
    total_steps += steps

print(f"step/episodio: {total_steps / episodies}")
print(f"penalita'/episodio: {total_penalties / episodies}")

step/episodio: 1.0
penalita'/episodio: 0.0
